<a href="https://colab.research.google.com/github/Lorddickenstein/FSLRwithNLP/blob/main/Application/NLP/CFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [531]:
import nltk
from nltk import Tree
from subprocess import *
grammar1 = nltk.CFG.fromstring("""
  S -> QP | VP SP | QP SP | O VP | WP | O JJ 
  SP -> PRP NN | PRP IN | NN | PRP |
  QP -> WP NN | WRB A2 | WP | WRB
  O -> SP A1 | SP A2 A3 | SP
  VP -> VB RB | VB WRB | VB TO | QP
  A1 -> "is"
  A2 -> "are"
  A3 -> "am"
  IN -> "from"
  JJ -> "good"
  WP -> "what" | "when"
  WDT -> "which"
  WRB -> "where" | "how"
  NN -> "name" | "okay" | "office" | "work"
  PRP -> "you" | "i"
  RB -> "here" 
  TO -> "to"
  VB -> "live" | "go"
  """)

grammar9 = nltk.CFG.fromstring("""
  S -> QP | SP VP | SP JJ | SP PNN | VP PP | SP | SP PP
  QP -> SP WQ | SP PP WQ | SP VP WQ
  SP -> PRP NN | PRP | NN | NN SP
  VP -> VB RB | VB 
  PP -> IN SP | IN 
  WQ -> WP | WRB
  NN -> 'name' | 'egg' | 'office' | 'work' | 'person' | 'student' 
  PRP -> 'you' | 'i-me' | 'he-she'
  WP -> 'what' | 'who' | 'when'
  WRB -> 'how' | 'where'
  JJ -> 'good' | 'okay'
  IN -> 'from' | 'to'
  RB -> 'here'
  VB -> 'live' | 'cook' | 'go' | 'study'
  NNP -> letter NNP | letter
  letter -> 'J' | 'E' | 'R' | 'S'
  PNN -> 'jers'
""")
# (S -> SP VP) (SP -> NN SP) (SP -> PRP) (VP -> VB) 
grammar10 = nltk.CFG.fromstring("""
  S -> QP | SP VP | SP JJ | SP NNP | VP PP |
  QP -> WQ SP | WQ SP PP | WQ SP VP 
  SP -> PRPS NN | PRP FWA | NN | NN SP | PRP | PRPS NN FWA 
  VP -> VB RB | VB NN | VB
  PP -> IN DT SP | IN SP | IN
  WQ -> WP FWA | WRB | WRB FWC | WRB FWA
  FWA -> 'is' | 'are' | 'am'
  FWC -> 'do' 
  NN -> 'name' | 'egg' | 'office' | 'work'
  PRPS -> 'your' | 'my'
  PRP -> 'you' | 'i-me' | 'he-she'
  WP -> 'what' | 'who' | 'when'
  WRB -> 'how' | 'where'
  JJ -> 'good' | 'okay'
  IN -> 'from' | 'to'
  RB -> 'here'
  VB -> 'live' | 'cook' | 'go'
  DT -> 'the' | 'an' | 'a'
  NNP -> 'jers' 
""")

# (S (QP (WP What)) (SP (PRP you) (NN name)))
grammar2 = nltk.CFG.fromstring("""
  S -> QP | VP SP | QP A1 SP | O SP | O VBP
  SP -> NN | PRP NN 
  QP ->  WP
  WP -> "what"
  A1 -> "is"
  NN -> "name"
  PRP -> "your"
  """)

grammar4 = nltk.CFG.fromstring("""
  S -> A
  A -> QP SP  
  SP -> NN
  QP -> WP O
  O -> PRP
  WP -> "what"
  PRP -> "your"
  NN -> "name"
  A1 -> "is"
  """)

grammar7= nltk.CFG.fromstring("""
  S -> A
  A -> QP SP  
  SP -> NN
  QP -> WP O
  O -> PRP
  WP -> "what"
  PRP -> "your"
  NN -> "name"
  A1 -> "is"
  """)
 	
grammar3 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)

grammar6 = nltk.CFG.fromstring("""
  S -> NP VP
  PP -> P NP
  NP -> Det N | Det N PP
  VP -> V NP | VP PP
  Det -> 'DT'
  N -> 'NN'
  V -> 'VBZ'
  P -> 'PP'
  """)
# What's your name (you name what?) - (S (O (SP (PRP you) (NN name))) (VBP (QP (WP what))))
# I'm good (I good) - (S (O (SP (PRP i))) (JJ good))
# Where you from (you from where) -> (S (O (SP (PRP you) (IN from))) (VBP (QP (WRB where))))
# I live here -> (S (O (SP (PRP i))) (VBP (VB live) (RB here)))
# How are you (How you?) -> (S (QP (WRB how)) (SP (PRP you)))
# Where do you live? (you, live, where?) -> (S (O (SP (PRP you))) (VBP (VB live) (WRB where)))
# Are you okay? (you, okay) ->(S (O (SP (PRP you) (NN okay))))
# My name is (I-Me, name) -> (S (O (SP (PRP i) (NN name))))
# Go to the Office (Go, to, Office) -> (S (VP (VB go) (TO to)) (SP (NN office)))
# Go to work (Go, to, work) -> (S (VBP (VB go) (TO to)) (SP (NN work)))

In [561]:
data_list = ['name', 'egg','office','work','person','student','you','i-me','he-she','what','who','when','how','where','good','okay','from','to','here','live',
             'cook','go','study']


def update_grammar(word='lagoon'):
  grammar = f"""
    S -> QP | SP VP | SP JJ | SP NNP | VP PP | SP
    QP -> SP WQ | SP PP WQ | SP VP WQ
    SP -> PRP NN | PRP | NN | NN SP 
    VP -> VB RB | VB 
    PP -> IN SP | IN
    WQ -> WP | WRB
    NN -> 'name' | 'egg' | 'office' | 'work' | 'person' | 'student' 
    PRP -> 'you' | 'i-me' | 'he-she'
    WP -> 'what' | 'who' | 'when'
    WRB -> 'how' | 'where'
    JJ -> 'good' | 'okay'
    IN -> 'from' | 'to'
    RB -> 'here'
    VB -> 'live' | 'cook' | 'go' | 'study' | 'come'
    NNP -> {word} 
  """

  return grammar

In [ ]:
grammar1.productions()

[S -> QP,
 S -> VP SP,
 S -> QP SP,
 S -> O VBP,
 S -> WP,
 S -> O JJ,
 S -> VBP SP,
 SP -> PRP NN,
 SP -> PRP IN,
 SP -> NN,
 SP -> PRP,
 SP -> ,
 QP -> WP NN,
 QP -> WRB A2,
 QP -> WP,
 QP -> WRB,
 O -> SP A1,
 O -> SP A2 A3,
 O -> SP,
 VBP -> VB RB,
 VBP -> VB WRB,
 VBP -> VB TO,
 VBP -> QP,
 A1 -> 'is',
 A2 -> 'are',
 A3 -> 'am',
 IN -> 'from',
 JJ -> 'good',
 WP -> 'what',
 WP -> 'when',
 WDT -> 'which',
 WRB -> 'where',
 WRB -> 'how',
 NN -> 'name',
 NN -> 'okay',
 NN -> 'office',
 NN -> 'work',
 PRP -> 'you',
 PRP -> 'i',
 RB -> 'here',
 TO -> 'to',
 VB -> 'live',
 VB -> 'go']

In [ ]:
import random


d = {}
 
d["s"]=[["a", "b"]]
d["a"]=[["what", "e"],["when", "e"]]
d["b"]=[["c", "d"]]
d["c"]=[["you"]]
d["d"]=[["name"],["bday"]]
d["e"]=[["is"]]

expansion = []
seed = "s"


def expand(start, expansion):
    if start in d:
    #Grab one possible expansion
        possible = d[start]
        random_expansion = random.choice(possible)
    # Call this method again with the current element
        for i in range(len(random_expansion)):
            expand(random_expansion[i], expansion);
 
  # if the rule wasn't found, then it's a terminal:
  # append the string to the expansion
    else:
        expansion.append(start)

expand(seed, expansion)

print(expansion)

['when', 'is', 'you', 'bday']


In [ ]:
' +|[A-Za-z]+|[()]'

In [324]:
import re

class ParseError(Exception):
    pass

# Tokenize a string.
# Tokens yielded are of the form (type, string)
# Possible values for 'type' are '(', ')' and 'WORD'
def tokenize(s, reg_ex):
    toks = re.compile(reg_ex)
    for match in toks.finditer(s):
        s = match.group(0)
        if s[0] == ' ':
            continue
        if s[0] in '()':
            yield (s, s)
        else:
            yield ('word', s)


# Parse once we're inside an opening bracket.
def parse_inner(toks):
    ty, name = next(toks)
    if ty != 'word': raise ParseError
    children = []
    while True:
        ty, s = next(toks)
        if ty == '(':
            children.append(parse_inner(toks))
        elif ty == ')':
            return (name, children)

# Parse this grammar:
# ROOT ::= '(' INNER
# INNER ::= WORD ROOT* ')'
# WORD ::= [A-Za-z]+
def parse_root(toks):
    ty, _ = next(toks)
    if ty != '(': raise ParseError
    return parse_inner(toks)

# def show_children(tree):
#     name, children = tree
#     if not children: return
#     print ("%s -> %s" % (name, ' '.join(child[0] for child in children)))
#     for child in children:
#         show_children(child)

# example = '(S (QP (SP (PRP you) (NN name)) (WQ (WP what))))'
# # show_children(parse_root(tokenize(example)))

In [ ]:
def show_children(tree):
    name, children = tree
    if not children: return
    print ("%s -> %s" % (name, ' '.join(child[0] for child in children)))
    for child in children:
        show_children(child)

In [281]:
def show_children(tree, pattern):
    name, children = tree
    if not children: return ""
    pattern = "(%s -> %s) " % (name, ' '.join(child[0] for child in children))
    for child in children:
      pattern += show_children(child, pattern)
    # return children, ("(%s -> %s) " % (name, ' '.join(child[0] for child in children)))
    return pattern



S -> QP
QP -> SP WQ
SP -> PRP NN
WQ -> WP

['PRP you', 'NN name',]


In [ ]:
('S ', [('QP ', [('SP ', [('PRP you', []), ('NN name', [])]), ('WQ ', [('WP what', [])])])])

In [248]:
def get_terminal(tree, terminals):
  name, children = tree
  # print(children)
  if not children:
    # print(name)
    terminals.append(name)
    return terminals
  for child in children:
    terminals = get_terminal(child, terminals)
  return terminals

  

In [305]:
from nltk.util import flatten
t = Tree.fromstring("(S (QP (WQ (WP what) (FWA is)) (SP (PRPS your) (NN name))))")
print(flatten(t))
' '.join(flatten(t))

['what', 'is', 'your', 'name']


'what is your name'

In [557]:
from nltk.util import flatten

def gen_sentence(terminals, pattern):
  tree = ""
  string = "unrecognized"

  # you name what
  if pattern == '(S -> QP) (QP -> SP WQ) (SP -> PRP NN) (WQ -> WP)':
    prps, nn, wp = terminals
    prps = 'your' if prps.split(' ')[1] == 'you' else 'my' if prps.split(' ')[1] == 'i-me' else 'his-her' 
    string = f'(S (QP (WQ (WP {wp.split()[1]}) (FWA is)) (SP (PRPS {prps}) (NN {nn.split()[1]}))))'
  # you live where
  elif pattern == '(S -> QP) (QP -> SP VP WQ) (SP -> PRP) (VP -> VB) (WQ -> WRB)':
    prp, vb, wrb = terminals
    fwc = 'does' if prp.split(' ')[1] == 'he-she' else 'do' 
    string = f'(S (QP (WQ (WRB {wrb.split()[1]}) (FWC {fwc})) (SP (PRP {prp.split()[1]})) (VP (VB {vb.split()[1]}))))'
  # you from where
  elif pattern == '(S -> QP) (QP -> SP PP WQ) (SP -> PRP) (PP -> IN) (WQ -> WRB)':
    prp, ins, wrb = terminals
    prp = prp.split(' ')[1]
    fwa = 'am' if prp == 'i-me' else 'are' if prp == 'you' else 'is'
    string = f'(S (QP (WQ (WRB {wrb.split()[1]}) (FWA {fwa})) (SP (PRP {prp})) (PP (IN {ins.split()[1]}))))'
  # i-me name +
  elif pattern == '(S -> SP NNP) (SP -> PRP NN)':
    prps, nn, nnp = terminals
    prps = 'your' if prps.split(' ')[1] == 'you' else 'my' if prps.split(' ')[1] == 'i-me' else 'his-her' 
    string = f'(S (SP (PRPS {prps}) (NN {nn.split()[1]}) (FWA is)) (NNP {nnp.split()[1]}))'
  # i-me good
  elif pattern == '(S -> SP JJ) (SP -> PRP)':
    prp, jj = terminals
    fwa = 'am' if prp.split(' ')[1] == 'i-me' else 'are' if prp.split(' ')[1] == 'you' else 'is'
    string = f'(S (SP (PRP {prp.split()[1]}) (FWA {fwa})) (JJ {jj.split()[1]}))'
  # egg i-me cook
  elif pattern == '(S -> SP VP) (SP -> NN SP) (SP -> PRP) (VP -> VB)':
    nn, prp, vb = terminals
    fwa = 'am' if prp.split(' ')[1] == 'i-me' else 'are' if prp.split(' ')[1] == 'you' else 'is'
    vb = vb.split(' ')[1] 
    prp = prp.split(' ')[1]
    vb = vb if prp == 'you' or prp == 'i-me' else vb + 's'
    string = f'(S (SP (PRP {prp})) (VP (VB {vb}) (NN {nn.split()[1]})))'
  # go to office
  elif pattern == '(S -> VP PP) (VP -> VB) (PP -> IN SP) (SP -> NN)':
    vb, ins, nn = terminals
    string = f'(S (VP (VB {vb.split()[1]})) (PP (IN {ins.split()[1]}) (SP (NN {nn.split()[1]}))))'

  tree = Tree.fromstring(string)
  return ' '.join(flatten(tree))



In [ ]:
where am i from
(S (QP (WQ (WRB where) (FWA am)) (SP (PRP i-me)) (PP (IN from))))

where are you from
(S (QP (WQ (WRB where) (FWA are)) (SP (PRP you)) (PP (IN from))))

where is he-she from
(S (QP (WQ (WRB where) (FWA is)) (SP (PRP he-she)) (PP (IN from))))

(S (SP (PRP i-me) (FWA am)) (JJ good))

In [554]:
text = "go to office"
sent = text.split()
# rd_parser = nltk.parse.shiftreduce.ShiftReduceParser(grammar6, trace=3)
rd_parser = nltk.parse.recursivedescent.RecursiveDescentParser(grammar10)
for tree in rd_parser.parse(sent):
  print(type(tree))
  print(tree)

<class 'nltk.tree.Tree'>
(S (VP (VB go)) (PP (IN to) (SP (NN office))))


In [569]:
tree = None
text = "go to person"
sent = text.split()

pnn_list = [word for word in sent if word not in data_list]
pnn_list = '\'' + '\' | \''.join(pnn_list) + '\'' if len(pnn_list) else ''
grammar = nltk.CFG.fromstring(update_grammar(word=pnn_list))

# rd_parser = nltk.parse.shiftreduce.ShiftReduceParser(grammar6, trace=3)
rd_parser = nltk.parse.recursivedescent.RecursiveDescentParser(grammar)
for tree in rd_parser.parse(sent):
  # print(tree)
  tree = tree
if tree:
  tree = parse_root(tokenize(str(tree),' +|[A-Z a-z -]+|[()]'))
  pattern = show_children(parse_root(tokenize(str(tree), ' +|[A-Za-z-]+|[()]')), "")
  print(tree)

  terminals = get_terminal(tree,[])

  sentence = gen_sentence(terminals, pattern.strip())
  print(sentence)
else:
  print("unrecognized")
# for node in tree:
#   print(node)

('S ', [('VP ', [('VB go', [])]), ('PP ', [('IN to', []), ('SP ', [('NN person', [])])])])
go to person


In [469]:
text = "Joshua Ella Jerson"
sent = text.split()
pnn_list = [word for word in sent if word not in data_list]
pnn_list = '\'' + '\' | \''.join(pnn_list) + '\''
grammar10 = nltk.CFG.fromstring(update_grammar(word=pnn_list))
print(grammar)

Grammar with 45 productions (start state = S)
    S -> QP
    S -> SP VP
    S -> SP JJ
    S -> SP PNN
    S -> VP PP
    S -> SP
    QP -> SP WQ
    QP -> SP PP WQ
    QP -> SP VP WQ
    SP -> PRP NN
    SP -> PRP
    SP -> NN
    SP -> NN SP
    VP -> VB RB
    VP -> VB
    PP -> IN SP
    PP -> IN
    WQ -> WP
    WQ -> WRB
    NN -> 'name'
    NN -> 'egg'
    NN -> 'office'
    NN -> 'work'
    NN -> 'person'
    NN -> 'student'
    PRP -> 'you'
    PRP -> 'i-me'
    PRP -> 'he-she'
    WP -> 'what'
    WP -> 'who'
    WP -> 'when'
    WRB -> 'how'
    WRB -> 'where'
    JJ -> 'good'
    JJ -> 'okay'
    IN -> 'from'
    IN -> 'to'
    RB -> 'here'
    VB -> 'live'
    VB -> 'cook'
    VB -> 'go'
    VB -> 'study'
    NNP -> 'Joshua'
    NNP -> 'Ella'
    NNP -> 'Jerson'


In [552]:
print(f'pattern {pattern.strip()}')
print(f'terminals {terminals}')

pattern (S -> VP PP) (VP -> VB) (PP -> IN SP) (SP -> NN)
terminals ['VB go', 'IN to', 'NN office']


In [ ]:
# for tree in tree:
#   print(length)
#   length = len(tree)
#   while length >= 0:
#     length = length - 1
#     for tree in tree:      
#       print(tree)

  # for tree in tree:
  #   for tree in tree:
  #     for tree in tree:
  #       print(tree)

  # list = (tree)
  # print(list)
  # print(list[0])
# nltk.parse.shiftreduce.demo()
# nltk.parse.recursivedescent.demo()

In [ ]:
from textblob import TextBlob

text = "Get ball"

blob = TextBlob(text)
blob.tags           # [('The', 'DT'), ('titular', 'JJ'),
                    #  ('threat', 'NN'), ('of', 'IN'), ...]

[('Get', 'VB'), ('the', 'DT'), ('ball', 'NN')]

In [ ]:
nltk.download('treebank')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True